# Importing Libraries

In [1]:
import nbimporter
from Processing import red_tages_extracted
from FHB_traits_output import trait_output_red_markers
from FHB_traits_output import trait_output_without_red_markers


import numpy as np  
import pandas as pd  
from skimage import io, color, measure, img_as_ubyte, morphology  
from skimage.io import imread  
from skimage.draw import polygon, line  
from skimage.measure import label, regionprops  
from skimage.morphology import square, closing, binary_dilation, opening, erosion, remove_small_objects, disk  
from skimage import exposure, filters 
from skimage.util import img_as_ubyte  
import cv2 
from ultralytics import YOLO  
import matplotlib.pyplot as plt  
from matplotlib import patches 
from PIL import Image  
import os  
from scipy.spatial.distance import euclidean, cdist  
from sklearn.preprocessing import StandardScaler  
import joblib  

In [2]:
def predict_disease_label(Area_value, Length_value, model_data):
    
    # Extract the standardized features (X_scaled), cluster labels, and the scaler used during training
    X_scaled, clusters, scaler = model_data['X_scaled'], model_data['clusters'], model_data['scaler']
    
    # Standardize the input data (CumSxArea_Region and CumSxLen_Region) to match the scale of the training data
    data = np.array([Area_value, Length_value]).reshape(1, -1)
    new_features_scaled = scaler.transform(data)

    # Calculate the Euclidean distance between the new data point and the existing scaled feature points
    # and find the closest cluster to the new data
    closest_cluster = np.argmin(cdist(new_features_scaled, X_scaled, metric='euclidean'))

    # Map the cluster number to a corresponding disease label (e.g., HR, MR, MS, HS)
    labels_map = {0: "HR", 3: "MR", 2: "MS", 1: "HS"}
    
    # Return the predicted disease label based on the closest cluster
    # If the cluster is not found in the labels map, return "Unknown"
    return labels_map.get(clusters[closest_cluster], "Unknown")

In [3]:
# Clustering model
model_data = joblib.load("Model_save/Hierarchical clustering model.joblib") 
model = YOLO("Model_save/Yolov8-WFD.pt") 

# Evaluating resistance level(With of red markers)

Single image

In [4]:
image_path="Data/2.jpg"
SxSpkPct_Plot,SxSpkPct_ROI, CumSxArea_ROI, AvgSxArea_Spk, CumSxLen_ROI, AvgSxLen_Spk,_,_=trait_output_red_markers(image_path,model)
predicted_labels = predict_disease_label(CumSxArea_ROI, CumSxLen_ROI, model_data)
data = {
    "SxSpkPct_Plot": [SxSpkPct_Plot],
    "SxSpkPct_ROI": [SxSpkPct_ROI],
    "CumSxArea_ROI": [CumSxArea_ROI],
    "AvgSxArea_Spk": [AvgSxArea_Spk],
    "CumSxLen_ROI": [CumSxLen_ROI],
    "AvgSxLen_Spk": [AvgSxLen_Spk],
    "Resistance_Level": [predicted_labels]
}

df = pd.DataFrame(data)
display(df)


image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 144.8ms
Speed: 0.0ms preprocess, 144.8ms inference, 73.2ms postprocess per image at shape (1, 3, 640, 352)


,SxSpkPct_Plot,SxSpkPct_ROI,CumSxArea_ROI,AvgSxArea_Spk,CumSxLen_ROI,AvgSxLen_Spk,Resistance_Level
0,0.2,0.102041,4.701468,0.856923,5.67572,0.86015,MR


Batch processing

In [5]:
input_folder = r"Data" 
results_data = []
# Iterate over all images in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    results = model.predict(image_path, conf=0.5, save=False, show=False, show_labels=False)
    # Process the diseased spike region and extract various metrics
    SxSpkPct_Plot,SxSpkPct_ROI, CumSxArea_ROI, AvgSxArea_Spk, CumSxLen_ROI, AvgSxLen_Spk,_,_=trait_output_red_markers(image_path,model)
    
    # Predict the disease resistance level based on the extracted metrics
    predicted_labels = predict_disease_label(CumSxArea_ROI, CumSxLen_ROI, model_data)
    results_data.append([image_name, SxSpkPct_Plot,SxSpkPct_ROI, CumSxArea_ROI, AvgSxArea_Spk, CumSxLen_ROI, AvgSxLen_Spk, predicted_labels])
# Convert results into a DataFrame
results_df = pd.DataFrame(results_data, columns=["Image name", "SxSpkPct_Plot","SxSpkPct_ROI", "CumSxArea_ROI", "AvgSxArea_Spk", "CumSxLen_ROI", "AvgSxLen_Spk", "Resistance level"])

# Save results to Excel
output_excel_path = r"Result\Results_with of red markers.xlsx"
results_df.to_excel(output_excel_path, index=False)

# Optionally, display a message
print(f"Results saved to {output_excel_path}")


image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\1.jpg: 640x320 2 scabs, 103 spikes, 139.3ms
Speed: 2.0ms preprocess, 139.3ms inference, 10.4ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\1.jpg: 640x320 2 scabs, 103 spikes, 10.5ms
Speed: 2.0ms preprocess, 10.5ms inference, 4.9ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 8.9ms
Speed: 2.2ms preprocess, 8.9ms inference, 3.0ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 10.1ms
Speed: 1.0ms preprocess, 10.1ms inference, 3.4ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\3.jpg: 640x320 1 scab, 105 spikes, 9.0ms
Speed: 3.0ms preprocess, 9.0ms inference, 4.3ms po

# Evaluating resistance level(Without of red markers)

Single image

In [6]:
image_path="Data/2.jpg"
SxSpkratio_Plot, CumSxArea_Plot, AvgSxArea_Spk, CumSxLen_Plot, AvgSxLen_Spk,_,_=trait_output_without_red_markers(image_path,model)
predicted_labels = predict_disease_label(CumSxArea_Plot, CumSxLen_Plot, model_data)
data = {
    "SxSpkratio_Plot": [SxSpkratio_Plot],
    "SxSpkPct_ROI": ["Nan"],
    "CumSxArea_Plot": [CumSxArea_Plot],
    "AvgSxArea_Spk": [AvgSxArea_Spk],
    "CumSxLen_Plot": [CumSxLen_Plot],
    "AvgSxLen_Spk": [AvgSxLen_Spk],
    "Resistance_Level":[predicted_labels]
}

df = pd.DataFrame(data)
display(df)


image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 39.5ms
Speed: 4.0ms preprocess, 39.5ms inference, 15.7ms postprocess per image at shape (1, 3, 640, 352)


,SxSpkratio_Plot,SxSpkPct_ROI,CumSxArea_Plot,AvgSxArea_Spk,CumSxLen_Plot,AvgSxLen_Spk,Resistance_Level
0,0.071429,Nan,4.284613,0.856923,4.300752,0.86015,MR


Batch processing

In [7]:
input_folder = r"Data" 
results_data = []
# Iterate over all images in the folder
for image_name in os.listdir(input_folder):
    image_path = os.path.join(input_folder, image_name)
    results = model.predict(image_path, conf=0.5, save=False, show=False, show_labels=False)
    # Process the diseased spike region and extract various metrics
    SxSpkratio_Plot, CumSxArea_Plot, AvgSxArea_Spk, CumSxLen_Plot, AvgSxLen_Spk,_,_=trait_output_without_red_markers(image_path,model)
    
    # Predict the disease resistance level based on the extracted metrics
    predicted_labels = predict_disease_label(CumSxArea_ROI, CumSxLen_ROI, model_data)
    results_data.append([image_name,SxSpkratio_Plot, CumSxArea_Plot, AvgSxArea_Spk, CumSxLen_Plot, AvgSxLen_Spk, predicted_labels])
# Convert results into a DataFrame
results_df = pd.DataFrame(results_data, columns=["Image name", "SxSpkratio_Plot","CumSxArea_Plot", "AvgSxArea_Spk", "CumSxLen_Plot", "AvgSxLen_Spk","Resistance level"])

# Save results to Excel
output_excel_path = r"Result\Results_without of red markers.xlsx"
results_df.to_excel(output_excel_path, index=False)

# Optionally, display a message
print(f"Results saved to {output_excel_path}")


image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\1.jpg: 640x320 2 scabs, 103 spikes, 26.4ms
Speed: 4.5ms preprocess, 26.4ms inference, 47.7ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\1.jpg: 640x320 2 scabs, 103 spikes, 35.0ms
Speed: 3.4ms preprocess, 35.0ms inference, 31.8ms postprocess per image at shape (1, 3, 640, 320)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 23.4ms
Speed: 3.0ms preprocess, 23.4ms inference, 31.7ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\2.jpg: 640x352 5 scabs, 65 spikes, 34.3ms
Speed: 6.0ms preprocess, 34.3ms inference, 24.4ms postprocess per image at shape (1, 3, 640, 352)

image 1/1 C:\Users\liush\Desktop\Evaluation plantform-wheat FHB\Data\3.jpg: 640x320 1 scab, 105 spikes, 17.0ms
Speed: 3.3ms preprocess, 17.0ms inference, 15.